In [1]:
!pip install PyPDF2 langchain==0.3.9 langchain-community==0.3.9 langchain-core==0.3.21 langchain-experimental==0.3.3 huggingface-hub==0.23.2 instructor==1.7.0 InstructorEmbedding==1.0.1 sentence-transformers==2.2.2 torch pdfplumber faiss-cpu


In [6]:
!pip install langchain_ollama==0.2.0

  Using cached langchain_ollama-0.2.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached ollama-0.4.4-py3-none-any.whl.metadata (4.7 kB)
Using cached langchain_ollama-0.2.0-py3-none-any.whl (14 kB)
Using cached ollama-0.4.4-py3-none-any.whl (13 kB)
  Attempting uninstall: ollama
    Found existing installation: ollama 0.2.0
    Uninstalling ollama-0.2.0:
      Successfully uninstalled ollama-0.2.0


In [2]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
import os
from langchain.embeddings import OllamaEmbeddings
import pdfplumber
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langchain_ollama.llms import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain





c:\Users\L01257675\Documents\Agosto-Diciembre2021-DELL\Ago-Diciembre2024\CADI2025\CADI-LLMs\.venv\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


The following function loads the pdf file and returns the vectorstore. Do not modify. You can run it with the command:

load_db(embeddings,pdf_path)

where "embeddings" and "pdf_path" have been previously defined.

In [3]:
## Do not mofify
def load_db(embeddings,pdf_path):
    text =''
    with open(pdf_path,'rb') as file:
        pdf_reader = PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()

    text_splitter=SemanticChunker(
        embeddings, breakpoint_threshold_type="percentile")
    #text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_text(text)
    # define embedding
    vectorstore = FAISS.from_texts(docs, embeddings)
    return vectorstore

Define the embeddings. You can choose the embeddings you prefer, here we have the option of Instructor Embeddings from huggingface (not local) or Ollama embeddings (local).

In [4]:
embeddings = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-xl')
#embeddings = OllamaEmbeddings()from langchain.embeddings import OllamaEmbeddings

load INSTRUCTOR_Transformer
max_seq_length  512


c:\Users\L01257675\Documents\Agosto-Diciembre2021-DELL\Ago-Diciembre2024\CADI2025\CADI-LLMs\.venv\Lib\site-packages\sentence_transformers\models\Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related t

Define the name and path of the pdf file to load. Modify at your needs:

In [5]:
pdf_path = "PeterPan.pdf"

In the following piece of code we check whether the file with the embeddings already exists (stored in a directory called "faiss_index"). If the directory doesn't exists the embeddings are created calling the function "load_db" previously defined. The embeddings for the same pdf file can be created only once.
Do not modify.

In [6]:
#Do not modify 
import os
if not os.path.exists('faiss_index'):
    vectorstore=load_db(embeddings,pdf_path)
    vectorstore.save_local("faiss_index")
else:
    vectorstore = FAISS.load_local("faiss_index",embeddings=embeddings,allow_dangerous_deserialization=True)


Create the document retriever:

In [7]:
#Do not modify
retriever = vectorstore.as_retriever()

Personalize your prompt template:

In [8]:
template = """Eres un asistente para contestar preguntas del ususario sobre el contenido del file subido, que trata de la historia de Peter Pan. 
    Contesta siempre en español y agradece el usuario por preguntar. Si las preguntas son sobre otro tema, contesta que no puedes contestar.
    {context}
    Question: {question}
    Helpful Answer:"""
qa_prompt = ChatPromptTemplate.from_template(template)

Create the comnversation chain. The chain is a "ConversionalRetrievalChain" to take information only from the retriever (the pdf file). The llm is used only to interpret the questions and formulate the text of the answers. The chat memory is already included in the chain.


In [9]:
#Do not modify
llm = OllamaLLM(model='llama3')
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": qa_prompt}
)

C:\Users\L01257675\AppData\Local\Temp\ipykernel_16964\515640156.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [10]:
history = []

Loop to handle the chat:

In [11]:
def handle_conversation(conversation,history):
    print('Escribe "exit" para salir')
    while True:
        question = input('Tu: ')
        if question.lower() == 'exit':
            break
        print(question)
        result = conversation({"question": question}, {"chat_history": history})
        history.append((question, result["answer"]))
        print("Bot: ",result['answer'])

handle_conversation(conversation_chain,history)

Escribe "exit" para salir
Tu:Haz un resumen en 100 palabras de la historia


C:\Users\L01257675\AppData\Local\Temp\ipykernel_16964\1286419362.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = conversation({"question": question}, {"chat_history": history})


Bot:  Here's a summary of the story in 100 words:

The story begins with Peter Pan, a boy who refuses to grow up, visiting the Darling children and their sister Wendy. Peter takes Wendy and her brothers on an adventure to Neverland, where they encounter the Lost Boys, pirates, and fairies. However, Captain Hook and his sidekick Smee are determined to capture Peter and kill him. Along the way, Peter and Wendy form a close bond, and she joins him in Neverland. When Hook tries to attack them, Peter uses his quick thinking and cunning to outsmart the pirates and save the day.
Tu:Haz un resumen de la historia, escribe en español por favor
Bot:  La historia comienza con Peter Pan, un niño que vive en el País de Nunca Jamás, donde siempre es joven y no tiene que obedecer a nadie. Un día, Peter llega al mundo humano y se encuentra con una niña llamada Wendy, quien le cuenta que su sombra no se la quedaba pegada. Juntos, Wendy ayuda a Peter a coserle su sombra.

Peter le propone a Wendy ir con 